---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [332]:
import pandas as pd
import re

doc = []
with open('Resources/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc, columns=['text'])
df['date'] = 0
#df['sorter'] = 0

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #print(df)

def date_sorter(df):
    df['text'] = df['text'].str.replace('Janaury', 'January')
    df['text'] = df['text'].str.replace('Decemeber', 'December')   

    for idx in df.index:    
        text = df['text'].loc[idx].replace('.','')

        # Line 0~124 (125 entries)
        reg_result = re.findall(r'\d{1,2}[/-]\d{1,2}[/-]\d{2,4}', text)   
        if((df['date'].loc[idx] == 0) & (len(reg_result) > 0)):
            df['date'].loc[idx] = pd.Timestamp(reg_result[0])

        # Line 125~193 (69 entries)
        # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
        if(idx > 124 & idx < 194):          
            reg_result = re.findall(r'(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:. )?(?:, )?\d{2,4}', text)   
            if((df['date'].loc[idx] == 0) & (len(reg_result) > 0)):
                if len(reg_result[0].split(' ')) == 3:
                    df['date'].loc[idx] = pd.Timestamp(reg_result[0])
        
        # Line 194~227 (34 entries)
        # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009 
        if(idx > 193 & idx < 228):
            reg_result = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2}, )?(?:\d{1,2}. )?\d{2,4}', text)   
            if((df['date'].loc[idx] == 0) & (len(reg_result) > 0)):
                if len(reg_result[0].split(' ')) == 3:
                    df['date'].loc[idx] = pd.Timestamp(reg_result[0])
        
        # Line 228~342 (115 entries)
        # Mar 2009, Mar, 2009 
        if(idx > 227 & idx < 343):
            reg_result = re.findall(r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*(?:,)? \d{2,4}', text)   
            if((df['date'].loc[idx] == 0) & (len(reg_result) > 0)):
                df['date'].loc[idx] = pd.Timestamp('1 ' + reg_result[0])

        # Line 343~454 (112 entries)
        # 3/2009  
        if(idx > 342 & idx < 455):
            reg_result = re.findall(r'\d{1,2}[/-]\d{2,4}', text)   
            if((df['date'].loc[idx] == 0) & (len(reg_result) > 0)):
                df['date'].loc[idx] = pd.Timestamp(reg_result[0])

        # Line 455~499 (45 entries)
        # 2009  
        if(idx > 454):
            reg_result = re.findall(r'\d{4}', text)   
            if((df['date'].loc[idx] == 0) & (len(reg_result) > 0)):
                df['date'].loc[idx] = pd.Timestamp('1/1/' + reg_result[0])

    #df.to_csv('df_dates.csv', encoding='utf-8')
    
    df = df.sort_values(by='date')
    
    df = df.reset_index()
    
    print(list(df['index']))
    
    return #df
date_sorter(df)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #print(date_sorter(df))

C:\Users\Marin\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


[9, 84, 2, 53, 28, 474, 153, 13, 129, 98, 111, 225, 31, 171, 191, 486, 335, 415, 36, 405, 323, 422, 375, 380, 345, 57, 481, 436, 104, 299, 162, 154, 402, 95, 73, 108, 156, 332, 182, 82, 351, 278, 214, 155, 223, 473, 49, 317, 11, 319, 40, 418, 165, 370, 382, 3, 50, 363, 219, 465, 237, 23, 342, 204, 258, 315, 27, 93, 17, 488, 303, 283, 395, 309, 419, 123, 19, 117, 232, 72, 189, 369, 493, 318, 239, 148, 105, 336, 6, 200, 81, 65, 434, 164, 378, 313, 495, 424, 398, 5, 254, 296, 75, 167, 21, 259, 499, 347, 150, 78, 340, 441, 361, 267, 221, 466, 39, 134, 197, 355, 430, 80, 444, 246, 85, 215, 263, 74, 403, 458, 16, 25, 127, 454, 70, 44, 59, 103, 112, 429, 88, 179, 470, 358, 205, 397, 294, 137, 295, 35, 438, 247, 209, 61, 107, 285, 175, 99, 455, 24, 275, 421, 48, 426, 489, 136, 30, 274, 10, 178, 1, 447, 280, 185, 228, 135, 69, 492, 199, 352, 8, 276, 230, 334, 96, 38, 368, 404, 261, 168, 29, 437, 423, 54, 284, 485, 68, 32, 349, 41, 63, 416, 55, 130, 116, 76, 462, 330, 37, 390, 256, 216, 174, 180